In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer
from time import time
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix

from sklearn.neural_network import MLPRegressor

In [8]:
data = pd.read_csv('data/historical_data1_2008/historical_data1_Q12008/historical_data1_Q12008_withflag.csv',
                       sep=',', 
                       header=0, 
                       names = [
                                'CREDIT_SCORE',
                                'FIRST_PAYMENT_DATE',
                                'FIRST_TIME_HOMEBUYER_FLAG',
                                'NUMBER_OF_UNITS',
                                'OCCUPANCY_STATUS',
                                'ORIGINAL_INCOME_RATIO',
                                'ORIGINAL_UPB',
                                'ORIGINAL_LTV',
                                'ORIGINAL_INTEREST_RATE',
                                'CHANNEL',
                                'PRODUCT_TYPE',
                                'PROPERTY_STATE',
                                'PROPERTY_TYPE',
                                'LOAN_PURPOSE',
                                'ORIGINAL_LOAN_TERM',
                                'NUMBER_OF_BORROWERS',
                                'DEFAULT_FLAG'
                    ])

771    200803
729    200805
769    200803
755    200803
760    200804
        ...  
751    200805
810    200803
753    200805
789    200804
792    200805
Name: CREDIT_SCORE, Length: 398439, dtype: int64
